In [ ]:
#---#| default_exp model.ms2

In [ ]:
#| hide
%reload_ext autoreload
%autoreload 2

# MS2 prediction

pDeepModel and its interface

In [ ]:
from peptdeep.model.ms2 import *

In [ ]:
#| hide
torch.manual_seed(1337)
sequence = 'AAAAAAAA'
max_instrument_num = 4
mod_feature_size=len(model_const['mod_elements'])
aa_indices = torch.LongTensor(get_batch_aa_indices([sequence]))
charges = torch.tensor([[2]])
instrument_indices = torch.LongTensor([1])
NCEs = torch.tensor([[0.3]])
mod_x = torch.zeros((1, aa_indices.size(1), mod_feature_size))
msms_model = ModelMS2Transformer(2, 1, mask_modloss=False)
msms_model.eval()
out = msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)
assert out.size() == (1,len(sequence)-1,2)
out

tensor([[[-0.0770,  0.0803],
         [-0.0555,  0.0817],
         [-0.0159, -0.0468],
         [ 0.0321, -0.2095],
         [ 0.0602, -0.3224],
         [ 0.0463, -0.3438],
         [-0.0423, -0.4887]]], grad_fn=<SliceBackward>)

In [ ]:
#| hide
msms_model = ModelMS2Transformer(2, 1, mask_modloss=True)
msms_model.eval()
out = msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)
assert out.size() == (1,len(sequence)-1,2)
out

tensor([[[0.1752, 0.0000],
         [0.1774, 0.0000],
         [0.1738, 0.0000],
         [0.1604, 0.0000],
         [0.1426, 0.0000],
         [0.1178, 0.0000],
         [0.0648, 0.0000]]], grad_fn=<SliceBackward>)

In [ ]:
#| hide
torch.manual_seed(1337)
sequence = 'AAAAAAAA'
max_instrument_num = 4
mod_feature_size=len(model_const['mod_elements'])
aa_indices = torch.LongTensor(get_batch_aa_indices([sequence]))
charges = torch.tensor([[2]])
instrument_indices = torch.LongTensor([1])
NCEs = torch.tensor([[0.3]])
mod_x = torch.zeros((1, aa_indices.size(1), mod_feature_size))
msms_model = ModelMS2Bert(2, 1, mask_modloss=False)
msms_model.eval()
out = msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)
assert out.size() == (1,len(sequence)-1,2)
msms_model.output_attentions = True
msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)
(
    len(msms_model.attentions), msms_model.attentions[0].size(),
    len(msms_model.modloss_attentions), msms_model.modloss_attentions[0].size()
)

(4, torch.Size([1, 8, 10, 10]), 1, torch.Size([1, 8, 10, 10]))

In [ ]:
#| hide
sequence = 'ACDEFGIK'
max_instrument_num = 4
mod_feature_size=len(model_const['mod_elements'])
aa_indices = torch.LongTensor(get_batch_aa_indices([sequence]))
charges = torch.tensor([[2]])
instrument_indices = torch.LongTensor([1])
NCEs = torch.tensor([[0.3]])
mod_x = torch.zeros((1, aa_indices.size(1), mod_feature_size))
msms_model = ModelMS2pDeep(2, 1, mask_modloss=False)
msms_model.eval()
out = msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)
assert out.size() == (1,len(sequence)-1,2)
out

tensor([[[-0.2764, -0.3128],
         [-0.2750, -0.3725],
         [-0.2712, -0.4207],
         [-0.2656, -0.4509],
         [-0.2563, -0.4414],
         [-0.2405, -0.4114],
         [-0.2136, -0.3333]]], grad_fn=<SliceBackward>)


### Examples

In [ ]:
fragment_intensity_df = pd.DataFrame({
    'b':np.arange(100)/100, 
    'y_modloss':np.arange(100)/100
})
repeat = 10
precursor_df = pd.DataFrame({
    'sequence': ['AGHCEWQMKYR']*repeat,
    'mods': ['Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidation@M']*repeat,
    'mod_sites': ['0;4;8']*repeat,
    'nAA': [11]*repeat,
    'nce': [20]*repeat,
    'instrument': 'QE',
    'charge': np.arange(1,repeat+1),
    'frag_start_idx':np.arange(10, dtype=int)*10,
    'frag_stop_idx':np.arange(10, dtype=int)*10+10,
})
precursor_df

,sequence,mods,mod_sites,nAA,nce,instrument,charge,frag_start_idx,frag_stop_idx
0,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,1,0,10
1,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,2,10,20
2,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,3,20,30
3,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,4,30,40
4,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,5,40,50
5,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,6,50,60
6,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,7,60,70
7,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,8,70,80
8,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,9,80,90
9,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,90,100


In [ ]:
fragment_intensity_df

,b,y_modloss
0,0.00,0.00
1,0.01,0.01
2,0.02,0.02
3,0.03,0.03
4,0.04,0.04
...,...,...
95,0.95,0.95
96,0.96,0.96
97,0.97,0.97
98,0.98,0.98


### Train an MSMS model

In [ ]:
pdeep = pDeepModel()
pdeep.charged_frag_types=['b','y_modloss']
pdeep.build(
    ModelMS2pDeep,
    num_frag_types = 2,
    num_modloss_types = 1,
    dropout=0.2,
    lr=1e-3
)
pdeep.train(precursor_df, epoch=5, batch_size=3, fragment_intensity_df=fragment_intensity_df)
pdeep.train_with_warmup(precursor_df, epoch=5, batch_size=3, warmup_epoch=3, fragment_intensity_df=fragment_intensity_df)

### Predict from the model

In [ ]:
predict_inten_df = pdeep.predict(precursor_df,reference_frag_df=fragment_intensity_df)
predict_inten_df.max()

b            1.0
y_modloss    0.0
dtype: float32

### Metrics

In [ ]:
_frag_inten_df = fragment_intensity_df.copy()
_frag_inten_df.iloc[0,1] = 0.22

In [ ]:
df, metrics_describ = calc_ms2_similarity(
    precursor_df, fragment_intensity_df, 
    _frag_inten_df, ['b'],
    spc_top_k=0
)
assert df.SPC.values[0] == 1
df, metrics_describ = calc_ms2_similarity(
    precursor_df, fragment_intensity_df, 
    _frag_inten_df, ['b','y_modloss'],
    spc_top_k=2
)
assert df.SPC.values[0] == -1
precursor_df

,sequence,mods,mod_sites,nAA,nce,instrument,charge,frag_start_idx,frag_stop_idx,PCC,COS,SA,SPC
0,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,1,0,10,0.248708,0.735389,0.526001,-1.0
1,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,2,10,20,1.000000,1.000000,1.000000,1.0
2,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,3,20,30,1.000000,1.000000,1.000000,1.0
3,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,4,30,40,1.000000,1.000000,1.000000,1.0
4,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,5,40,50,1.000000,1.000000,1.000000,1.0
5,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,6,50,60,1.000000,1.000000,1.000000,1.0
6,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,7,60,70,1.000000,1.000000,1.000000,1.0
7,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,8,70,80,1.000000,1.000000,1.000000,1.0
8,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,9,80,90,1.000000,1.000000,1.000000,1.0
9,AGHCEWQMKYR,Acetyl@Protein_N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,90,100,1.000000,1.000000,1.000000,1.0


In [ ]:
metrics_describ

,PCC,COS,SA,SPC
count,10.000000,10.000000,10.000000,10.000000
mean,0.924871,0.973539,0.952600,0.800000
std,0.237579,0.083677,0.149892,0.632455
min,0.248708,0.735389,0.526001,-1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000
>0.90,0.900000,0.900000,0.900000,0.900000
>0.75,0.900000,0.900000,0.900000,0.900000


In [ ]:
pdeep.grid_nce_search(precursor_df, fragment_intensity_df)

(15, 'Lumos')

In [ ]:
pdeep = pDeepModel(mask_modloss=False)
pdeep.get_parameter_num()

3988974